## Generate Tezos Keys from Ledger Nano S Mnemonic Phrase

This notebook explores how to the Ledger nano S generates key hashes from recovery phrases. The Ledger adheres to BIP39 and BIP32.

In [1]:
# import tools
import bip_utils as bip
import pytezos
from pytezos.crypto.encoding import base58_decode, base58_encode, scrub_input
import hmac
import hashlib as hl
import pysodium
from pyblake2 import blake2b
import base58

In [2]:
# This is the seed phrase and Tezos public key hash (address) produced by a Ledger and the Tezos wallet app.
# The Base58 encoding (with prefixes) for the public key and private key that generates this address are also given.
phrase = 'motor jazz team food when coach reward hidden obtain faculty tornado crew toast inhale purchase conduct cube omit illness carbon ripple thank crew material'
pkh = 'tz1MN2TrBeSeTvj7Lu3RxeqRwvTcBBo98iza'
pk = 'edpkthCnqGYzzKZa3YFJYDDafa5xnZxdQQsHXh8n7qwFu1JRpRAE94'
# private key
secret = 'edskRvZKHaBdkQH8tkJZRVUF6kkSdXpEdBhJGFJh2fj9EgyEsAj2fTFNCUrBb2R33rwFgV9MLaqftUsSMqR5PqCL6omkmBMwBC'
# this is the default derivation path used by the Tezos wallet on the Ledger
default_path = "m/44'/1729'/0'/0'"

In [3]:
# use pytezos module to convert the private key to hex
secret_key = scrub_input(secret)
sk = base58_decode(secret_key)
print(sk.hex())

8dbdbca8b45da2953a1a487aa89fda6addda80e006bc6ccf876ff22814ab7f4806c45aa41971c6d2459367defc0ef76078bc0240b56471aab4305a8f26e67484


In [4]:
# what does pytezos do with the encoded secret
pytezos.Key.from_encoded_key(secret.encode('utf-8'))


Public key hash
tz1MN2TrBeSeTvj7Lu3RxeqRwvTcBBo98iza

Helpers
.blinded_public_key_hash()
.from_alias()
.from_encoded_key()
.from_faucet()
.from_mnemonic()
.from_public_point()
.from_secret_exponent()
.generate()
.public_key()
.public_key_hash()
.secret_key()
.sign()
.verify()

In [5]:
# use bib_utils module to see if the Ledger uses a standard implementation of BIP32
entropy = bip.Bip39MnemonicDecoder().Decode(phrase) # phrase to entropy
seed = bip.Bip39SeedGenerator(phrase).Generate() # entropy to seed
print(seed.hex())

bip32_ctx = bip.Bip32Ed25519Slip.FromSeed(seed) # uses Ed25519 curve like Tezos tz1 accounts
bip32_ctx = bip32_ctx.ChildKey(bip.Bip32Utils.HardenIndex(44)).ChildKey(bip.Bip32Utils.HardenIndex(1729)).ChildKey(bip.Bip32Utils.HardenIndex(0)).ChildKey(bip.Bip32Utils.HardenIndex(0))
sk_bip32 = bip32_ctx.PrivateKey().Raw().ToBytes()
print(sk_bip32.hex())

# try another way
bip32_ctx = bip.Bip32Ed25519Slip.FromSeedAndPath(seed, default_path) # uses Ed25519 curve like Tezos tz1 accounts
sk_bip32 = bip32_ctx.PrivateKey().Raw().ToBytes()
print(sk_bip32.hex())

pk_bip32 = bip32_ctx.PublicKey().RawCompressed().ToBytes()
print(pk_bip32.hex())

secret_exponent_bip32 = sk_bip32 + pk_bip32[1:]
print(secret_exponent_bip32.hex())


accecb88d83f6fe670361e31dcf853f688d1b3f7882e9e5e3367b66e637eb15b8a80066eb1b5b495bd55c48fd30717b5a877a66192f05a9f64cd50b38c41e7eb
8dbdbca8b45da2953a1a487aa89fda6addda80e006bc6ccf876ff22814ab7f48
8dbdbca8b45da2953a1a487aa89fda6addda80e006bc6ccf876ff22814ab7f48
0006c45aa41971c6d2459367defc0ef76078bc0240b56471aab4305a8f26e67484
8dbdbca8b45da2953a1a487aa89fda6addda80e006bc6ccf876ff22814ab7f4806c45aa41971c6d2459367defc0ef76078bc0240b56471aab4305a8f26e67484


In [6]:
# what does pytezos do with this sk
print(len(sk))
pytezos.Key.from_secret_exponent(sk)

64



Public key hash
tz1MN2TrBeSeTvj7Lu3RxeqRwvTcBBo98iza

Helpers
.blinded_public_key_hash()
.from_alias()
.from_encoded_key()
.from_faucet()
.from_mnemonic()
.from_public_point()
.from_secret_exponent()
.generate()
.public_key()
.public_key_hash()
.secret_key()
.sign()
.verify()

In [7]:
print(len(secret_exponent_bip32))
pytezos.Key.from_secret_exponent(secret_exponent_bip32)

64



Public key hash
tz1MN2TrBeSeTvj7Lu3RxeqRwvTcBBo98iza

Helpers
.blinded_public_key_hash()
.from_alias()
.from_encoded_key()
.from_faucet()
.from_mnemonic()
.from_public_point()
.from_secret_exponent()
.generate()
.public_key()
.public_key_hash()
.secret_key()
.sign()
.verify()

### Can I recode these results?

In [20]:
def mnemonic_to_seed(phrase:str, password:str = ''):
    passphrase = 'mnemonic' + password
    seed = hl.pbkdf2_hmac('sha512', phrase.encode('utf-8'), passphrase.encode('utf-8'), 2048)
    return seed

def parse_path(path: str): # default tezos path
    if path.endswith('/'):
        path = path[:-1]
    path = path.split('/')
    
    if path[0] == 'm' or path[0] == '':
        path = path[1:]
    
    indeces = []
    for each in path:
        if each.endswith("'"): # hardened
            index = (1 << 31) + int(each[:-1])
            index = index.to_bytes(4,'big') # from int to bytes
        else:
            index = int(each)
        indeces.append(index)
    
    return indeces

def hmac_sha512(key, data):
    I = (hmac.new(key, data, hl.sha512)).digest()
    IL = I[:32]
    IR = I[32:]
    return IL, IR

def seed_to_master(seed):
    data = seed
    key = b'ed25519 seed'
    master_sk, master_cc = hmac_sha512(key, data)
    return master_sk, master_cc

def parent_to_child(parent_sk, parent_cc, index):
    data = b'\x00' + parent_sk + index
    child_sk, child_cc = hmac_sha512(parent_cc, data)
    return child_sk, child_cc

def derivation_path_to_keys(master_sk, master_cc, path:str = "m/44'/1729'/0'/0'"):
    indeces = parse_path(path)
    child_sk = master_sk
    child_cc = master_cc
    for index in indeces:
        child_sk, child_cc = parent_to_child(child_sk, child_cc, index)
    
    return child_sk, child_cc
    
def sk_to_public_point(sk): # for Ed25519, the public point is the public key and the secret exponent is the sk || pk
    public_point, secret_exponent = pysodium.crypto_sign_seed_keypair(seed=sk)
    return public_point, secret_exponent
    
def pk_hash(public_point, prefix: bytes): # default prefix is tz1
    pkh = blake2b(data=public_point, digest_size=20).digest()
    tzaddress = blake2b_checksum(pkh, prefix)
    return tzaddress

def blake2b_checksum(data: bytes, prefix: bytes):
    b58 = base58.b58encode_check(prefix + data)
    return b58
    

In [25]:
# SLIP10 - alternative to BIP32
seed = mnemonic_to_seed(phrase) # mnemonic to seed
master_sk, master_cc = seed_to_master(seed) # seed to master secret key and chaincode 
child_sk, child_cc = derivation_path_to_keys(master_sk, master_cc, "m/44'/1729'/0'/0'") # master sk and cc to child sk and cc
public_point, secret_exponent = sk_to_public_point(child_sk) # public point and secret exponent
tzaddress = pk_hash(public_point, prefix = b'\x06\xa1\x9f') # prefix b'tz1'
secret = blake2b_checksum(secret_exponent, prefix = b'+\xf6N\x07') # prefix b'edsk'
pk = blake2b_checksum(public_point, prefix = b'\r\x0f%\xd9')
print(tzaddress)
print(secret)
print(pk)

b'tz1MN2TrBeSeTvj7Lu3RxeqRwvTcBBo98iza'
b'edskRvZKHaBdkQH8tkJZRVUF6kkSdXpEdBhJGFJh2fj9EgyEsAj2fTFNCUrBb2R33rwFgV9MLaqftUsSMqR5PqCL6omkmBMwBC'
b'edpkthCnqGYzzKZa3YFJYDDafa5xnZxdQQsHXh8n7qwFu1JRpRAE94'


In [9]:
# SLIP10 - alternative to BIP32

# 1) master key from seed
data = seed
key = b'ed25519 seed'
I = hmac.new(key, data, hl.sha512)
master_sk = I.digest()[:32] # IL
master_cc = I.digest()[32:] # IR

# 2) child 44'
I = hmac.new(master_cc, b'\x00' + master_sk + ((0x01 << 31)+44).to_bytes(4,'big'), hl.sha512)
child_cc = I.digest()[32:]
child_sk = I.digest()[:32]

# 3) child 1729'
I = hmac.new(child_cc, b'\x00' + child_sk + ((0x01 << 31) + 1729).to_bytes(4,'big'), hl.sha512)
child_cc = I.digest()[32:]
child_sk = I.digest()[:32]

# 4) child 0'
I = hmac.new(child_cc, b'\x00' + child_sk + ((0x01 << 31) + 0).to_bytes(4,'big'), hl.sha512)
child_cc = I.digest()[32:]
child_sk = I.digest()[:32]

# 5) child 0'
I = hmac.new(child_cc, b'\x00' + child_sk + ((0x01 << 31) + 0).to_bytes(4,'big'), hl.sha512)
child_cc = I.digest()[32:]
child_sk = I.digest()[:32]

public_point, secret_exponent = pysodium.crypto_sign_seed_keypair(seed=child_sk)
print(child_sk.hex())
print(secret_exponent.hex())
print(public_point.hex())

8dbdbca8b45da2953a1a487aa89fda6addda80e006bc6ccf876ff22814ab7f48
8dbdbca8b45da2953a1a487aa89fda6addda80e006bc6ccf876ff22814ab7f4806c45aa41971c6d2459367defc0ef76078bc0240b56471aab4305a8f26e67484
06c45aa41971c6d2459367defc0ef76078bc0240b56471aab4305a8f26e67484


In [23]:
# Blake2b hash and base58 encoding
def tb(l):
    return b''.join(map(lambda x: x.to_bytes(1, 'big'), l))


pkh = blake2b(data=public_point, digest_size=20).digest()
prefix = tb([6, 161, 159]) # tz1
tzaddress = base58.b58encode_check(prefix + pkh)
print(tzaddress)

b'tz1MN2TrBeSeTvj7Lu3RxeqRwvTcBBo98iza'


In [53]:
tb([43, 246, 78, 7])

b'+\xf6N\x07'

In [24]:
tb([13, 15, 37, 217])

b'\r\x0f%\xd9'

In [17]:
e = bip.Bip39MnemonicDecoder().Decode(phrase)
s = bip.Bip39SeedGenerator(phrase).Generate()
print(e.hex())
print(s.hex())

906eef7aad6fa2596e2b5b988a379599ae30e76b897635734dc4913ba5bf8cd4
accecb88d83f6fe670361e31dcf853f688d1b3f7882e9e5e3367b66e637eb15b8a80066eb1b5b495bd55c48fd30717b5a877a66192f05a9f64cd50b38c41e7eb


In [32]:
bip32_ctx = bip.Bip32Ed25519Slip.FromSeed(s)
bip32_ctx = bip32_ctx.ChildKey(bip.Bip32Utils.HardenIndex(44)).ChildKey(bip.Bip32Utils.HardenIndex(1729)).ChildKey(bip.Bip32Utils.HardenIndex(0)).ChildKey(bip.Bip32Utils.HardenIndex(0))

In [33]:
bip32_ctx.PrivateKey().Raw().ToBytes().hex()

'8dbdbca8b45da2953a1a487aa89fda6addda80e006bc6ccf876ff22814ab7f48'

In [40]:
bip32_ctx.PublicKey().RawUncompressed().ToBytes().hex()

'0006c45aa41971c6d2459367defc0ef76078bc0240b56471aab4305a8f26e67484'

In [2]:
import pytezos
pytezos.Key.from_encoded_key(secret.encode('utf-8'))


Public key hash
tz1MN2TrBeSeTvj7Lu3RxeqRwvTcBBo98iza

Helpers
.blinded_public_key_hash()
.from_alias()
.from_encoded_key()
.from_faucet()
.from_mnemonic()
.from_public_point()
.from_secret_exponent()
.generate()
.public_key()
.public_key_hash()
.secret_key()
.sign()
.verify()

In [3]:
from pytezos.crypto.encoding import base58_decode, base58_encode, scrub_input

In [4]:
secret_key = scrub_input(secret)

In [5]:
secret_key

b'edskRvZKHaBdkQH8tkJZRVUF6kkSdXpEdBhJGFJh2fj9EgyEsAj2fTFNCUrBb2R33rwFgV9MLaqftUsSMqR5PqCL6omkmBMwBC'

In [6]:
secret_decode = base58_decode(secret_key)
print(secret_decode.hex())
print(len(secret_decode))

8dbdbca8b45da2953a1a487aa89fda6addda80e006bc6ccf876ff22814ab7f4806c45aa41971c6d2459367defc0ef76078bc0240b56471aab4305a8f26e67484
64


* pkh = 'tz1bVaVSGQYcwZF1Qfc5zTbqghEhncigcsPX' when using root /ed25519 key
* pkh = 'tz3MHwDevGRmfUF11zkpG5XtYpPWGdRgNtdM' when using /P-256/0h/0h
* pkh = 'tz3TsZ7fQyCSpf16MVYttxDELJgGAW83oc6Z' when using /P-256

In [7]:
# test bip32
from bip32 import BIP32, HARDENED_INDEX

In [9]:
from mnemonic import Mnemonic
import hashlib as hl
import numpy as np
import pysodium
import binascii
import struct
import hmac
import ecdsa
from ecdsa.curves import SECP256k1
from ecdsa.ecdsa import int_to_string, string_to_int

In [10]:
mnemo = Mnemonic('english')
entropy = mnemo.to_entropy(phrase)
seed = Mnemonic.to_seed(phrase)
print(entropy.hex())
print(seed.hex())

906eef7aad6fa2596e2b5b988a379599ae30e76b897635734dc4913ba5bf8cd4
accecb88d83f6fe670361e31dcf853f688d1b3f7882e9e5e3367b66e637eb15b8a80066eb1b5b495bd55c48fd30717b5a877a66192f05a9f64cd50b38c41e7eb


In [11]:
secret = seed[:32]
chain = seed[32:]
print(secret.hex())
print(chain.hex())

accecb88d83f6fe670361e31dcf853f688d1b3f7882e9e5e3367b66e637eb15b
8a80066eb1b5b495bd55c48fd30717b5a877a66192f05a9f64cd50b38c41e7eb


In [16]:
# mnemonic phrase to seed and secret exponent
password = ''
mnemonic = phrase
passphrase = 'mnemonic' + password
# passphrase = '|'
mnemonic_bytes = mnemonic.encode("utf-8")
print(mnemonic_bytes)
passphrase_bytes = passphrase.encode("utf-8")
stretched = hl.pbkdf2_hmac("sha512", mnemonic_bytes, passphrase_bytes, 2048)
seed = stretched[:64]
print(seed.hex())
# for P256/secp256r1 secret exponent is the first 32 bytes of the seed, for 
secret_exponent = seed[:32]
print(secret_exponent.hex())

b'motor jazz team food when coach reward hidden obtain faculty tornado crew toast inhale purchase conduct cube omit illness carbon ripple thank crew material'
accecb88d83f6fe670361e31dcf853f688d1b3f7882e9e5e3367b66e637eb15b8a80066eb1b5b495bd55c48fd30717b5a877a66192f05a9f64cd50b38c41e7eb
accecb88d83f6fe670361e31dcf853f688d1b3f7882e9e5e3367b66e637eb15b


In [10]:
public_point, secret_exponent = pysodium.crypto_sign_seed_keypair(seed=secret_exponent)
# public_point, secret_exponent = pysodium.crypto_sign_seed_keypair(seed=bytes(entropy))
public_point2 = pysodium.crypto_sign_sk_to_pk(sk=seed)
# pk, sk = pysodium.crypto_sign_seed_keypair(bytes(entropy))
# pk2 = pysodium.crypto_sign_sk_to_pk(sk = sk)
# print(pk.hex())
# print(sk.hex())
# print(pk2.hex())

print(public_point.hex())
print(public_point2.hex())


4bf723052e2f624cf23b4dee2d7b00715e352b86cbbb27f2f286bc7f5e302a4b
8a80066eb1b5b495bd55c48fd30717b5a877a66192f05a9f64cd50b38c41e7eb


In [11]:
from pyblake2 import blake2b
from pytezos.crypto.encoding import base58_decode, base58_encode
# generate hash
pkh = blake2b(data=public_point2, digest_size=20).digest()
print(pkh.hex())
prefix = b'tz1'
public_hash = base58_encode(pkh, prefix).decode()
print(public_hash)



db61c95a1344a6d5f6a8cb40046f31ee239a539f
tz1fe1pTUgDkn5PGW9QgwCWQiAqpdq1CRAev


In [94]:
# add checksum
h = hl.sha256()
h.update(entropy)
entropy_cs = entropy
entropy_cs.append(h.digest()[0])
print(entropy_cs.hex())

906eef7aad6fa2596e2b5b988a379599ae30e76b897635734dc4913ba5bf8cd447


In [34]:
# convert byte array to bit array
bits = []
for byte in entropy_cs:
    for i in range(8):
        if (byte & 0b10000000 >> i):
            bits.append(1)
        else:
            bits.append(0)
        
print(bits)
print(len(bits))

# import vocab
# import the vocabulary
f = open('bip39english.txt', 'r')
vocab = [line.strip('\n') for line in f]
f.close()

# iterate through bits with 11 bit chunks
my_words = []
for k in range(0,int(len(bits)/11)):
    chunk = bits[k*11:k*11+11]
    index = 0
    for j in range(11):
        index += chunk[j]*2**(11-1-j)
    my_words.append(vocab[index])
my_phrase = ' '.join(my_words)
print(my_phrase)

[1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1]
264
motor jazz team food when coach reward hidden obtain faculty tornado crew toast inhale purchase conduct cube omit illness carbon ripple thank crew material


In [35]:
# generate public point from secret
public_point, secret_exponent = pysodium.crypto_sign_seed_keypair(seed=seed[:32])
print(secret_exponent.hex())
print(public_point.hex())

74f8315f8990058a25aa8b4b80b20afcd3b72b50dc3bed40cfdc6b9362f9cc3cfba44c0e12c41f474c7a772367bcf0af8cf213e47d507fdc66b9937aca0b7286
fba44c0e12c41f474c7a772367bcf0af8cf213e47d507fdc66b9937aca0b7286


In [ ]:
mnemo = Mnemonic("english")
phrase = mnemo.generate(256)
print(phrase)

In [ ]:
entropy = mnemo.to_entropy(phrase)
entropy_list = list(entropy)
print(entropy)
print(entropy_list)
print(entropy.hex())

In [ ]:
# calculate checksum length
el = len(entropy)*8 # length in bits
csl = int(el/32) # checksum length
print(csl)

In [ ]:
h = hl.sha256()
h.update(entropy)
entropy.append(h.digest()[0])
print(entropy.hex())

In [ ]:
# convert byte array to bit array
bits = []
for byte in entropy:
    for i in range(8):
        if (byte & 0b10000000 >> i):
            bits.append(1)
        else:
            bits.append(0)
        
print(bits)
print(len(bits))

In [ ]:
# import vocab
# import the vocabulary
f = open('bip39english.txt', 'r')
vocab = [line.strip('\n') for line in f]
f.close()

In [ ]:
# iterate through bits with 11 bit chunks
my_words = []
for k in range(0,int(len(bits)/11)):
    chunk = bits[k*11:k*11+11]
    index = 0
    for j in range(11):
        index += chunk[j]*2**(11-1-j)
    my_words.append(vocab[index])
my_phrase = ' '.join(my_words)
print(my_phrase)

## Entropy to Mnemonic

The following shows how to go from a random seed (entropy) to 24-word mnemonic phrase

In [ ]:
# generate 256 bit random number
myentropy = bytearray(np.random.bytes(32))
print(myentropy.hex())

In [ ]:
# add hash byte
h = hl.sha256()
h.update(myentropy)
myentropy.append(h.digest()[0])
print(myentropy.hex())

In [ ]:
# convert byte array to bit array
bits = []
for byte in myentropy:
    for i in range(8):
        if (byte & 0b10000000 >> i):
            bits.append(1)
        else:
            bits.append(0)
        
print(bits)
print(len(bits))

In [ ]:
# import vocab
# import the vocabulary
f = open('bip39english.txt', 'r')
vocab = [line.strip('\n') for line in f]
f.close()

In [ ]:
# iterate through bits with 11 bit chunks
my_words = []
for k in range(0,int(len(bits)/11)):
    chunk = bits[k*11:k*11+11]
    index = 0
    for j in range(11):
        index += chunk[j]*2**(11-1-j)
    my_words.append(vocab[index])
my_phrase = ' '.join(my_words)
print(my_phrase)

In [ ]:
# check
e = mnemo.to_entropy(my_phrase)
e_list = list(e)
print(e)
print(e_list)
print(e.hex())

## Mnemonic to Seed

Tezos takes the mnemonic and hashes it with an optional passphrase to formulate a secret key.

In [15]:
password = ''
mnemonic = phrase
passphrase = 'mnemonic' + password
passphrase = 'Bitcoin seed' + password
mnemonic_bytes = mnemonic.encode("utf-8")
print(mnemonic_bytes)
passphrase_bytes = passphrase.encode("utf-8")
stretched = hl.pbkdf2_hmac("sha512", mnemonic_bytes, passphrase_bytes, 2048)
seed = stretched[:64]
print(seed.hex())
# for P256/secp256r1 secret exponent is the first 32 bytes of the seed
secret_exponent = seed[:32]
print(secret_exponent.hex())

b'motor jazz team food when coach reward hidden obtain faculty tornado crew toast inhale purchase conduct cube omit illness carbon ripple thank crew material'
b8172558fa4ef06cecbbb6873c8c50989710f9f8c307375769fdb6de516d41447c6fe5a6da41919f4ce8dccc673e457c310a7f050d4edc76a0917d62fbd3ee06
b8172558fa4ef06cecbbb6873c8c50989710f9f8c307375769fdb6de516d4144


In [28]:
# P256
import fastecdsa.keys 
import fastecdsa.curve
from fastecdsa.encoding.util import bytes_to_int
import fastecdsa.encoding.sec1
from pyblake2 import blake2b
from pytezos.crypto.encoding import base58_decode, base58_encode

pk = fastecdsa.keys.get_public_key(bytes_to_int(secret_exponent), curve=fastecdsa.curve.P256)
public_point = fastecdsa.encoding.sec1.SEC1Encoder.encode_public_key(pk)
print(pk)
print(public_point.hex())

X: 0x6bdee78f49c3a11c6c8d495a2a7489d24494441218795d54f2a46fc48840373b
Y: 0x72935d82358185f6a793bbf3aa8afd0d1ebef78ad3196eed39710d894ff02b96
(On curve <P256>)
026bdee78f49c3a11c6c8d495a2a7489d24494441218795d54f2a46fc48840373b


In [29]:
pkh = blake2b(data=public_point, digest_size=20).digest()
print(pkh.hex())
prefix = b'tz3'
public_hash = base58_encode(pkh, prefix).decode()
print(public_hash)

bf3bad293c6e802a979047a3be576ce73f5c77a4
tz3dmCAh4DzjUJohjqFiuBJsmaUunomE3e8e


In [30]:
# compare to pytezos results
from pytezos.crypto.key import Key
p2pk = Key.from_public_point(
    bytes.fromhex(public_point.hex()), 
    curve=b'p2')
p2pk


Public key hash
tz3dmCAh4DzjUJohjqFiuBJsmaUunomE3e8e

Helpers
.blinded_public_key_hash()
.from_alias()
.from_encoded_key()
.from_faucet()
.from_mnemonic()
.from_public_point()
.from_secret_exponent()
.generate()
.public_key()
.public_key_hash()
.secret_key()
.sign()
.verify()

### Questions

How does a ledger convert seed phrase into keys and key hash? 